In [ ]:
%pip install -qqqU langchain-qdrant

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_aws import BedrockEmbeddings, ChatBedrock
from qdrant_client.http.models import Distance, VectorParams
from langchain_core.rate_limiters import InMemoryRateLimiter

In [ ]:
model_id = 'meta.llama3-8b-instruct-v1:0'

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.07,  # <-- Super slow! We can only make a request once every 10 seconds!!
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    # max_bucket_size=10,  # Controls the maximum burst size.
)

llm = ChatBedrock(
    model_id=model_id,
    model_kwargs={"temperature": 0},
    rate_limiter=rate_limiter
    )

embeddings = BedrockEmbeddings()

In [ ]:
import os

client = QdrantClient(
    location=os.environ["VECTOR_STORE_URL"],
    api_key=os.environ["VECTOR_STORE_API_KEY"]
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="regulamento-semantic",
    embedding=embeddings
)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = """
Você é um assistente da UFRN responsável por instruir alunos sobre questões acadêmicas do regulamento dos cursos de graduação.
Você deve responder a resposta correta baseada na questão e contexto abaixo. Por favor, siga as instruções:

1. Pergunta: {pergunta}

2. Contexto: {contexto}

3. Instruções:
    - Analise cuidadosamente a questão e o contexto fornecido.
    - Formule uma resposta abrangente e precisa baseada apenas nas informações fornecidas no contexto.
    - Certifique-se de que sua resposta aborda diretamente a pergunta.
    - Inclua todas as informações relevantes do contexto, mas não adicione nenhum conhecimento externo.
    - Se o contexto não contiver informações suficientes para responder completamente à pergunta, declare isso claramente e forneça a melhor resposta parcial possível.
    - Use um tom formal e objetivo.

"""

prompt_template = PromptTemplate.from_template(prompt)

In [ ]:
import pandas as pd

df = pd.read_csv('../data/dataset_potiguana.csv')
df.head(2)

In [ ]:
from tqdm import tqdm

retrieved_contexts = []
responses = []

for _, row in tqdm(df.iterrows()):
    query = row['user_input']
    context_docs = vector_store.similarity_search(query, k=2)
    contexts = [c.page_content for c in context_docs]
    prompt = prompt_template.format(contexto=contexts, pergunta=query)
    base_message = llm.invoke(prompt)
    response = base_message.content

    retrieved_contexts.append(contexts)
    responses.append(response)

In [ ]:
df['response'] = pd.Series(responses)
df['retrieved_contexts'] = pd.Series(retrieved_contexts)

df.head(2)

In [ ]:
df.to_csv('../naive_rag_llama3-8b_k2.csv')

In [ ]:
from datasets import Dataset

def pandas_to_ragas(df):
    '''
    Converts a Pandas DataFrame into a Ragas-compatible dataset
    
    Inputs:
        - df (Pandas DataFrame): The input DataFrame to be converted
        
    Returns:
        - ragas_testset (Hugging Face Dataset): A Hugging Face dataset compatible with the Ragas framework
    '''
    # Ensure all text columns are strings and handle NaN values
    text_columns = df.columns
    for col in text_columns:
        df[col] = df[col].fillna('').astype(str)
        
    # Convert 'contexts' to a list of lists
    df['reference_contexts'] = df['reference_contexts'].fillna('').astype(str).apply(lambda x: [x] if x else [])
    df['retrieved_contexts'] = df['retrieved_contexts'].fillna('').astype(str).apply(lambda x: [x] if x else [])
    
    # Converting the DataFrame to a dictionary
    data_dict = df.to_dict('list')
    
    # Loading the dictionary as a Hugging Face dataset
    ragas_testset = Dataset.from_dict(data_dict)
    
    return ragas_testset

eval_dataset = pandas_to_ragas(df)

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [ ]:
from ragas.metrics import (
    # ContextPrecision,
    LLMContextRecall,
    NonLLMContextRecall,
    ContextEntityRecall,
    ResponseRelevancy,
    Faithfulness,
    SemanticSimilarity,
)

metrics = [
    # ContextPrecision(llm=evaluator_llm),
    LLMContextRecall(llm=evaluator_llm),
    NonLLMContextRecall(),
    ContextEntityRecall(llm=evaluator_llm),
    ResponseRelevancy(llm=evaluator_llm),
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings),
]

In [ ]:
from ragas import evaluate

results = evaluate(dataset=eval_dataset, metrics=metrics)

In [ ]:
result_df = results.to_pandas()

In [ ]:
result_df.to_csv('../naive_rag_llama3-8b_k2_result.csv')